In [1]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt
from utils.detection.association_geodesic import squarize
plt.rcParams.update({'font.size': 18})

from utils.physics.signal.make_spectrogram import make_spectrogram
from utils.data_reading.sound_data.station import StationsCatalog
import skimage
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, HomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [10]:
# paths
CATALOG_PATH = "/media/plerolland/akoustik/MAHY"
DETECTIONS_DIR = f"../../../../../data/detection/TiSSNet_Pn"
OUT = "/media/plerolland/LaBoite/MAHY_eval/Pn"

STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()

DELTA = datetime.timedelta(seconds=100)

In [11]:
d = []
for station in tqdm(STATIONS):
    path = f"{DETECTIONS_DIR}/{station.dataset}/{station.dataset}_{station.name}.pkl"
    print(path)

    with open(path, "rb") as f:
        while True:
            try:
                d.append([station] + pickle.load(f))
            except EOFError:
                break
d = np.array(d)

print(len(d), d.shape)
print(d[0])

  0%|          | 0/19 [00:00<?, ?it/s]

../../../../../data/detection/TiSSNet_Pn/MAHY0/MAHY0_MAHY01.pkl


  5%|▌         | 1/19 [00:00<00:03,  5.91it/s]

../../../../../data/detection/TiSSNet_Pn/MAHY0/MAHY0_MAHY02.pkl


 16%|█▌        | 3/19 [00:00<00:03,  5.14it/s]

...

../../../../../data/detection/TiSSNet_Pn/MAHY2/MAHY2_MAHY22.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY2/MAHY2_MAHY23.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY3/MAHY3_MAHY31.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY3/MAHY3_MAHY32.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY3/MAHY3_MAHY33.pkl


100%|██████████| 19/19 [00:01<00:00, 17.26it/s]

../../../../../data/detection/TiSSNet_Pn/MAHY3/MAHY3_MAHY34.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY4/MAHY4_MAHY41.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY4/MAHY4_MAHY42.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY4/MAHY4_MAHY43.pkl
../../../../../data/detection/TiSSNet_Pn/MAHY4/MAHY4_MAHY44.pkl


100%|██████████| 19/19 [00:01<00:00, 10.63it/s]


226894 (226894, 3)
[MAHY0_MAHY01 datetime.datetime(2020, 10, 16, 0, 18, 47, 374469)
 np.float16(0.4172)]


In [12]:
d = d[d[:,2] >= 0.5]
print(len(d))
np.random.seed(0)
np.random.shuffle(d)
d = d[:1_000]

96103


In [14]:
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import skimage.io
import numpy as np


for s, date, p in tqdm(d):
    manager, date, delta, path = s.get_manager(), date, DELTA, f'{OUT}/{s.name}_{date.strftime("%Y%m%d_%H%M%S_%f")}.png'
    if date - delta < manager.dataset_start or date + delta > manager.dataset_end:
        print("skipping out of bounds segment")
        continue
    data = manager.get_segment(date - delta, date + delta)
    spectrogram = make_spectrogram(data,
                                   manager.sampling_f,
        t_res=0.25,
        f_res=2,
        return_bins=False,
        normalize=False
    ).astype(np.float32)

    # Normalisation
    spectrogram = (spectrogram - np.mean(spectrogram)) / np.std(spectrogram)
    spectrogram = 255 * (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Sauvegarde
    skimage.io.imsave(path, spectrogram.astype(np.uint8))

100%|██████████| 1000/1000 [01:59<00:00,  8.39it/s]


In [16]:
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import skimage.io
import numpy as np

def process_detection(args):
    manager, date, delta, path = args
    if Path(path).exists():
        return
    if date - delta < manager.dataset_start or date + delta > manager.dataset_end:
        #print("skipping out of bounds segment")
        return
    data = manager.get_segment(date - delta, date + delta)
    spectrogram = make_spectrogram(
        data,
        manager.sampling_f,
        t_res=0.25,
        f_res=2,
        return_bins=False,
        normalize=False
    ).astype(np.float32)

    # Normalisation
    spectrogram = (spectrogram - np.mean(spectrogram)) / np.std(spectrogram)
    spectrogram = 255 * (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Sauvegarde
    skimage.io.imsave(path, spectrogram.astype(np.uint8))

for s, date, p in detections_uncertain:
    process_detection((s.get_manager(), date, DELTA, f'{OUT}/{s.name}_{date.strftime("%Y%m%d_%H%M%S_%f")}.png'))


KeyboardInterrupt

